In [ ]:
# EV Routing pakcage
from ev_routing.floyd_warshall_profile import FloydWarshallProfile
# Visualization packages
import plotly.graph_objects as go
import os


# Initializing Dijsktra
bl = {'lat': 52.510e0, 'lon': 13.385e0} # Bottom left corner coordinate
tr = {'lat': 52.520e0, 'lon': 13.389e0} # Top right corner coordinate

M = 30000

area = [bl['lat'], bl['lon'], tr['lat'], tr['lon']]
fwp = FloydWarshallProfile(area, M)

# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']

latitudes = [v['lat'] for v in fwp.v.values()]
longitudes = [v['lon'] for v in fwp.v.values()]

node_ids = [v['id'] for v in fwp.v.values()]
node_labels = [str(i) for i in range(len(fwp.v))]

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = latitudes,
    lon = longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 12,
        color = 'rgb(242, 177, 172)',
        opacity=1.0
    ),
    text = node_labels,
    hoverinfo = 'text'
))

nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = fwp.map_center[0],
            lon = fwp.map_center[1]
        ),
        pitch = 0,
        zoom = 14,
        style = 'light'
    ),
)

nodes_map.update_layout(margin = {"r": 0, "t": 0, "l": 0, "b": 0})
nodes_map.show()

In [ ]:
import time

start_time = time.time()
history = fwp.run_with_history()
print("%s s" % (time.time() - start_time))

In [ ]:
dt = 6

for i in [125]:
    for j in [75]:
        elevation_map = go.Figure()

        def plot_break_points(tt, ii, jj):
            x, y, text = [], [], []
            for bp in history[tt][ii][jj]:
                x.append(bp[0])
                y.append(bp[1] if bp[1] >= 0 else -10)
                text.append(str(bp))

            elevation_map.add_trace(go.Scatter(x=x, y=y, text=text, mode='lines+markers'))

        for t in range(0, len(history), dt):
            plot_break_points(t, i, j)

        if len(history) % dt != 0:
            plot_break_points(-1, i, j)


        elevation_map.update_layout(
            margin = { "r": 0, "t": 0, "l": 0, "b": 0 },
            xaxis_title = 'Initial charge',
            yaxis_title = 'Final charge',
            xaxis_range = [-10, M+10],
            yaxis_range = [-20, M+10]
        )

        elevation_map.show()